In [1]:
import os
import pandas as pd
import xlwings as xw
import imgkit

def main():
    # Rutas a modificar:
    xlsm_file = r"C:\Users\Mayores\Documents\Python Scripts\Facturación Automatización LIBRES\Libres_Ensa_auto_python.xlsm"  # Cambia esta ruta por la de tu archivo Excel (.xlsm)
    prn_folder = r"C:\Users\Mayores\Documents\Python Scripts\Facturación Automatización LIBRES\PRN LIBRES ENE25"                 # Cambia esta ruta por la carpeta donde se encuentran tus archivos PRN

    # Carpeta para guardar las imágenes generadas (para revisión posterior)
    capture_folder = os.path.join(prn_folder, "capturas")
    os.makedirs(capture_folder, exist_ok=True)

    # Configuración explícita para wkhtmltoimage
    wkhtmltoimage_path = r"C:\Program Files\wkhtmltopdf\bin\wkhtmltoimage.exe"
    config = imgkit.config(wkhtmltoimage=wkhtmltoimage_path)

    # Opciones para la generación de la imagen
    options = {
        "format": "png",
        "encoding": "UTF-8"
    }

    # Abrir Excel en modo invisible para evitar mostrar la interfaz
    app = xw.App(visible=False)
    try:
        wb = app.books.open(xlsm_file)
        sheets = list(wb.sheets)

        # Se omite la primera hoja y las últimas 5 (si hay suficientes hojas)
        if len(sheets) <= 6:
            print("[ADVERTENCIA] El libro no tiene suficientes hojas para omitir la primera y las últimas 5.")
            sheets_to_process = []
        else:
            sheets_to_process = sheets[1:-5]

        # Procesar cada hoja
        for sheet in sheets_to_process:
            process_sheet(sheet, prn_folder, capture_folder, config, options)

        # Guardar los cambios en el archivo Excel
        wb.save()
        print(f"\nEl archivo Excel '{xlsm_file}' se ha actualizado con las imágenes.")
    except Exception as e:
        print("Error al procesar el archivo Excel:", e)
    finally:
        wb.close()
        app.quit()

def process_sheet(sheet, prn_folder, capture_folder, config, options):
    sheet_name = sheet.name
    print(f"\nProcesando hoja: '{sheet_name}'")

    # 1) Leer el número de suministro desde la celda E2
    supply_number = sheet.range("E2").value
    if not supply_number:
        print(f"  - La celda E2 está vacía en la hoja '{sheet_name}'. Se omite.")
        return

    try:
        # Convertir el número a entero y formatearlo a 8 dígitos (agregando ceros a la izquierda)
        supply_str = str(int(supply_number)).zfill(8)
    except ValueError:
        print(f"  - Valor no numérico en E2 en la hoja '{sheet_name}': {supply_number}. Se omite.")
        return

    # 2) Construir la ruta del archivo PRN correspondiente
    prn_file = os.path.join(prn_folder, f"{supply_str}.prn")
    if not os.path.exists(prn_file):
        print(f"  - El archivo PRN '{prn_file}' no existe. Se omite la hoja '{sheet_name}'.")
        return

    # 3) Leer el archivo PRN (asumiendo que el separador es TAB y codificación UTF-8)
    try:
        df = pd.read_csv(prn_file, sep='\t', encoding='utf-8')
    except Exception as e:
        print(f"  - Error al leer '{prn_file}': {e}")
        return

    if df.empty:
        print(f"  - El archivo PRN '{prn_file}' está vacío. Se omite la hoja '{sheet_name}'.")
        return

    # 4) Obtener las primeras 5 filas y convertirlas a una tabla HTML
    html_table = df.head(5).to_html(index=False)

    # 5) Generar la imagen a partir del HTML utilizando imgkit con la configuración explícita
    img_file = os.path.join(capture_folder, f"{supply_str}.png")
    try:
        imgkit.from_string(html_table, img_file, options=options, config=config)
        print(f"  - Imagen generada: '{img_file}'")
    except Exception as e:
        print(f"  - Error al generar la imagen para la hoja '{sheet_name}': {e}")
        return

    # 6) Insertar la imagen en la hoja Excel en el rango F7:J13 (se ajusta para ocupar el rango completo)
    try:
        target_range = sheet.range("F7:J13")
        left = target_range.left
        top = target_range.top
        width = target_range.width
        height = target_range.height
        sheet.pictures.add(img_file, left=left, top=top, width=width, height=height)
        print(f"  - Imagen insertada en la hoja '{sheet_name}' en el rango F7:J13.")
    except Exception as e:
        print(f"  - Error al insertar la imagen en la hoja '{sheet_name}': {e}")

if __name__ == "__main__":
    main()



Procesando hoja: 'Backus'
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
  - Imagen generada: 'C:\Users\Mayores\Documents\Python Scripts\Facturación Automatización LIBRES\PRN LIBRES ENE25\capturas\35707003.png'
  - Imagen insertada en la hoja 'Backus' en el rango F7:J13.

Procesando hoja: 'NICOLL '
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
  - Imagen generada: 'C:\Users\Mayores\Documents\Python Scripts\Facturación Automatización LIBRES\PRN LIBRES ENE25\capturas\25820885.png'
  - Imagen insertada en la hoja 'NICOLL ' en el rango F7:J13.

Procesando hoja: 'CINEPLANET'
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
  - Imagen generada: 'C:\Users\Mayores\Documents\P